# 2. Change data from raw to tabular format

## Import

In [1]:
import pandas as pd
import numpy as np
import re
import os

# Run preprocessing

In [2]:
HOUSE_TYPES = ['House', 'Vacant land', 'Apartment', 'Townhouse', 'Villa', 'Retirement Living',
               'Development site', 'Terrace', 'New house and land', 'Semi-detached', 'Duplex']

In [3]:
for files in ['rawdata1804202504', 'rawdata1804202505', 'rawdata1804202506']:
    with open('../'+files+'.txt', 'r') as f:
        lines = f.readlines()

    df_rows = []

    for line in lines:
        line_list = eval(line)

        for i in range(len(line_list)):

            house_info = {
                'address': None,
                'price': None,
                'sold_date': None,
                'sold_type': None,
                'bedrooms': None,
                'bathroom': None,
                'car_space': None,
                'land_size (m2)': None,
                'house_type': None,
            }

            # part 1: sale information
            sold_date_info = line_list[i][0]

            # SOLD DATE
            sold_date_info_split = sold_date_info.split(' ')
            house_info['sold_date'] = ' '.join(sold_date_info_split[-3:])

            if sold_date_info_split[2] == 'auction':
                house_info['sold_type'] = 'auction'
            elif sold_date_info_split[2] == 'private':
                house_info['sold_type'] = 'private treaty'
            elif sold_date_info_split[2] == 'to':
                house_info['sold_type'] = 'sold prior to auction'
            else:
                pass

            # part 2: title
            other_info = line_list[i][1]

            # PRICE
            house_info_str = other_info.split(' ')[0]
            house_info['price'] = int(
                house_info_str.replace('$', '').replace(',', ''))
            other_info = other_info.replace(house_info_str, '')

            # BEDROOMS
            bedroom_str_list = re.findall(
                r'(\d+) Bed(?=[s0-9])', other_info)  # use regex to find
            if len(bedroom_str_list) == 0:
                bedroom = 0
                house_info['bedrooms'] = bedroom
            else:
                bedroom_str = bedroom_str_list[0]
                for ending in ['Bed', 'Beds']:
                    other_info = other_info.replace(
                        f'{bedroom_str} {ending}', '')  # remove from other_info
                # split from postcode and convert to int
                bedroom = int(bedroom_str[4:])
                house_info['bedrooms'] = bedroom

            # BATHROOMS
            baths_str_list = re.findall(
                r'(\d+) Bath(?=[s0-9\u2212])', other_info)
            if len(baths_str_list) == 0:
                baths_str = 0
                house_info['bathroom'] = baths_str
            else:
                baths_str = baths_str_list[0]
                for ending in ['Bath', 'Baths']:
                    other_info = other_info.replace(
                        f'{baths_str} {ending}', '')  # remove from other_info
                bathroom = int(baths_str)
                house_info['bathroom'] = bathroom

            # CAR SPACE
            parking_str_list = re.findall(r'(\d+) Parking', other_info)
            if len(parking_str_list) == 0:
                parking_str_list = re.findall(r'(\u2212) Parking', other_info)
                if len(parking_str_list) != 0:
                    other_info = re.sub(r'\u2212 Parking', '', other_info)
                    house_info['car_space'] = 0

            else:
                other_info = other_info.replace(
                    f'{parking_str_list[0]} Parking', '')
                car_space = int(parking_str_list[0])
                house_info['car_space'] = car_space

            # LAND SIZE
            unit = 'm'
            if 'm²' in other_info:
                land_size_str_list = re.findall(r'(\d+)m²', other_info)
            elif 'Vacant land' in other_info:
                unit = 'ha'
                land_size_str_list = re.findall(
                    r'(\d+(?:\.\d+)?)ha', other_info)
            if len(land_size_str_list) == 0:
                pass
            else:
                land_size_str = land_size_str_list[0]
                other_info = other_info.replace(
                    f'{land_size_str}m²' if unit == 'm' else f'{land_size_str}ha', '')
                land_size = float(land_size_str)
                if unit == 'ha':
                    land_size = land_size * 10000
                house_info['land_size (m2)'] = land_size

            # HOUSE TYPE
            for house_type in HOUSE_TYPES:
                if house_type in other_info:
                    house_info['house_type'] = house_type
                    other_info = other_info.replace(house_type, '')
                    break

            # ADDRESS
            house_info['address'] = other_info.strip().replace('\xa0', '')

            df_rows.append(house_info)

    # Create DataFrame
    df = pd.DataFrame(df_rows)

    # Save DataFrame to CSV
    output_file = f'../data/curated/processed_{files}.csv'
    df.to_csv(output_file, index=False)

# Brief look at data

In [7]:
file_list = os.listdir('../data/curated/')

total_properties = 0
total_valid_houses = 0
for file in file_list:
    if file.endswith('.csv'):
        df = pd.read_csv('../data/curated/' + file)
        total_properties += len(df)
        total_valid_houses += len(df[(df['house_type'] == 'House')
                                     & (df['land_size (m2)'].notnull())])

print(f'Total number of properties: {total_properties}')
print(f'Total number of valid houses: {total_valid_houses}')

Total number of properties: 30314
Total number of valid houses: 25473
